### Student Information
Name:毛胤絜

Student ID:113062596

GitHub ID:

Kaggle name:

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__. 

In [2]:
### Begin Assignment Here

In [19]:
import os
import random
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import itertools
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import AdamW
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
from torch.utils.data.dataloader import default_collate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 加載資料並分成train和test

In [4]:
#identification_df = pd.read_csv("data_identification.csv")
#all_data = pd.read_json("tweets_DM.json", lines=True, encoding="utf-8")
#all_data_df = pd.DataFrame(all_data)
#emotion_df = pd.read_csv("emotion.csv")

In [5]:
#all_data_df['tweet_id'] = all_data_df['_source'].apply(lambda x: x['tweet']['tweet_id'])
#all_data_df['hashtags'] = all_data_df['_source'].apply(lambda x: x['tweet']['hashtags'])
#all_data_df['text'] = all_data_df['_source'].apply(lambda x: x['tweet']['text'])
#all_data_df = all_data_df.drop(columns=['_source'])

#print(all_data_df['tweet_id'])

# 將all_data_df和identification_df根據tweet_id做合併
#merged_df = pd.merge(all_data_df, identification_df, on='tweet_id', how='inner')

# 分離出train_df和test_df
#train_df = pd.merge(merged_df[merged_df['identification']=='train'], emotion_df, on='tweet_id', how='inner')
#train_df = train_df.drop(columns=['identification','_type','_index'])



#test_df = merged_df[merged_df['identification'] == 'test']
#test_df = test_df.drop(columns=['identification','_type','_index']) 

In [6]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

# 處理表情符號

In [7]:
emoji_pattern = re.compile(#由GPT生成
    "[\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251"  # Enclosed characters
    "]+", flags=re.UNICODE)

# emoji欄位儲存在text欄位中找到的表情符號
train_df['emojis'] = train_df['text'].apply(lambda x: emoji_pattern.findall(x))

In [8]:
# 把所有表情符號列表展平成一個單一的列表
all_emojis = list(itertools.chain(*train_df['emojis']))

# 拆開表情符號組合，並保留唯一的表情符號
unique_emojis = [emoji for sublist in all_emojis for emoji in sublist]  # 拆開表情符號，因為可能還保有連續的表情符號
unique_emojis = list(set(unique_emojis))  # 去除重複的表情符號

print(unique_emojis[0:30])# 包含哪些符號或是韓文、中文字



['처', '🌥', '😶', 'ﾟ', 'す', '認', '아', '能', '⚡', '✯', 'ﷺ', '♡', '片', '스', '\uf648', '름', '🤭', '🡐', 'モ', '🅔', '⚓', '𗂭', '문', '製', '🍌', '念', '🍔', '\ue00d', '╚', '🤠']


# 分析表情符號跟emotion的關係

In [9]:
# 整理出具有表情符號的部分
train_df_with_emojis = train_df[train_df['emojis'].apply(len) > 0]
train_df_with_emojis['emojis'] = train_df_with_emojis['emojis'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# 拆開所有 emoji(盡量避免連續的emoji)
all_emojis = list(itertools.chain(*train_df_with_emojis['emojis'].str.split()))

# 建立 emoji 和 emotion 的組合
emoji_emotion_data = []
for index, row in train_df_with_emojis.iterrows():
    emojis = row['emojis'].split()  # 以空格分割每行中的表情符號
    emotion = row['emotion']
    for emoji in emojis:
        emoji_emotion_data.append([emoji, emotion])

emoji_emotion_df = pd.DataFrame(emoji_emotion_data, columns=['emoji', 'emotion'])

# 統計每個emotion有多少個獨特的 emoji
emotion_emoji_counts = emoji_emotion_df.groupby('emotion')['emoji'].nunique().reset_index(name='unique_emoji_count')

# 顯示結果
print(emotion_emoji_counts)# 看得出明顯差別，雖然df中可能還是包含連續的表情符號，但是把它們看做一個表情符號邏輯上也可以接受



C:\Users\USER\AppData\Local\Temp\ipykernel_25508\3835693287.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_with_emojis['emojis'] = train_df_with_emojis['emojis'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)


        emotion  unique_emoji_count
0         anger                2269
1  anticipation               14525
2       disgust                5298
3          fear                4358
4           joy               37767
5       sadness                7879
6      surprise                3502
7         trust               15586


# emotion dic

In [10]:
emotion_to_id = {'anger': 0,
 'fear': 1,
 'disgust': 2,
 'joy': 3,
 'anticipation': 4,
 'trust': 5,
 'sadness': 6,
 'surprise': 7}
id_to_emotion = {v: k for k, v in emotion_to_id.items()}
emotion_to_id 

{'anger': 0,
 'fear': 1,
 'disgust': 2,
 'joy': 3,
 'anticipation': 4,
 'trust': 5,
 'sadness': 6,
 'surprise': 7}

# TF-IDF做words embedding

In [11]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=5, max_df=0.9, max_features=300,ngram_range=(1, 2)     )

tfidf_matrix = tfidf_vectorizer.fit_transform(train_df["text"])
train_tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
                             
tfidf_matrix = tfidf_vectorizer.fit_transform(test_df["text"])
test_tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

test_tfidf_df = test_tfidf_df.reindex(columns=train_tfidf_df.columns, fill_value=0)


# 添加emoji_count

In [12]:
def extract_emojis(text):
    # 使用正則表達式來提取表情符號
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+")
    return emoji_pattern.findall(text)

# 處理文本並計算每條文本的emoji數量
def preprocess_and_add_to_df(texts, batch_size):
    emoji_count_list = []  # 用來儲存每條文本中的 emoji 數量

    for i in tqdm(range(0, len(texts), batch_size), desc="處理中"):
        batch_texts = texts[i:i + batch_size]
        
        # 計算每條文本中的表情符號數量
        batch_emoji_counts = [len(extract_emojis(text)) for text in batch_texts]
        emoji_count_list.extend(batch_emoji_counts)

    return emoji_count_list

# 只處理 emoji_count
print("正在處理train文本...")
train_emoji_counts = preprocess_and_add_to_df(
    train_df['text'].tolist(), batch_size=16
)
train_df["emoji_count"] = train_emoji_counts  # 添加emoji數量

# 只處理 emoji_count
print("正在處理test文本...")
test_emoji_counts = preprocess_and_add_to_df(
    test_df['text'].tolist(), batch_size=16
)
test_df["emoji_count"] = test_emoji_counts  # 添加emoji數量

print("處理完成")


正在處理train文本...


處理中: 100%|█████████████████████████████████████████████████████████████████| 90973/90973 [00:06<00:00, 15062.85it/s]


正在處理test文本...


處理中: 100%|█████████████████████████████████████████████████████████████████| 25749/25749 [00:01<00:00, 13041.98it/s]

處理完成


# 合併TF-IDF和emoji_count

In [13]:
# 將 emoji_count 欄位與 TF-IDF 特徵合併
train_features = pd.concat([train_tfidf_df, train_df["emoji_count"].reset_index(drop=True)], axis=1)
test_features = pd.concat([test_tfidf_df, test_df["emoji_count"].reset_index(drop=True)], axis=1)

# 假設你的標籤是 "emotion" 欄位
train_labels = train_df["emotion"].map(lambda x: emotion_to_id[x]).reset_index(drop=True)

# Training

In [14]:
# 切分訓練集和驗證集（如果需要）
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.1, random_state=42)

# 初始化隨機森林模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 訓練模型
rf_model.fit(X_train, y_train)

# 驗證模型
val_predictions = rf_model.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, val_predictions)}")

Validation Accuracy: 0.4450627589191863


# Submission with classifier

In [17]:
from sklearn.metrics import f1_score
sample_df = pd.read_csv("sampleSubmission.csv")

# 使用訓練好的模型進行預測
test_predictions = rf_model.predict(test_features)

y_pred_labels = [id_to_emotion[pred] for pred in test_predictions]

test_df['y_pred_labels'] = y_pred_labels
test_df = test_df.rename(columns={'tweet_id': 'id'})
merged_df = sample_df.merge(test_df[['id', 'y_pred_labels']], on='id', how='inner')

# 獲取真實標籤和預測標籤
true_labels = merged_df['emotion']
pred_labels = merged_df['y_pred_labels']


submission_mean_f1 = f1_score(true_labels, pred_labels, average='macro')
print(f"Mean F1-Score: {submission_mean_f1:.4f}")
submission_accuracy = accuracy_score(true_labels, pred_labels)
print(f"Accuracy: {submission_accuracy:.4f}")

Mean F1-Score: 0.0019
Accuracy: 0.0077


# Submission with Regressor

In [20]:
# 回歸模型
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# 預測並計算均方誤差
test_predictions = regressor.predict(test_features)
y_pred_labels = [id_to_emotion[pred] for pred in test_predictions]

test_df['y_pred_labels'] = y_pred_labels
test_df = test_df.rename(columns={'tweet_id': 'id'})
merged_df = sample_df.merge(test_df[['id', 'y_pred_labels']], on='id', how='inner')

true_labels = merged_df['emotion']
pred_labels = merged_df['y_pred_labels']


submission_mean_f1 = f1_score(true_labels, pred_labels, average='macro')
print(f"Mean F1-Score: {submission_mean_f1:.4f}")
submission_accuracy = accuracy_score(true_labels, pred_labels)
print(f"Accuracy: {submission_accuracy:.4f}")

MemoryError: Unable to allocate 2.94 GiB for an array with shape (301, 1310006) and data type float64